## Get the Crypto data

In [32]:
# Import libraries and dependencies
import ccxt
import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv

In [33]:
load_dotenv()
kraken_public_key = os.getenv("KRAKEN_PUBLIC_KEY")
kraken_secret_key = os.getenv("KRAKEN_SECRET_KEY")
exchange = ccxt.kraken({"apiKey": kraken_public_key, "secret": kraken_secret_key})



In [34]:
# Fetch daily candlestick bar data from `BTC/USD`
historical_prices = exchange.fetch_ohlcv("BTC/USD", "1d")

# Import the data as a Pandas DataFrame and set the columns
historical_prices_df = pd.DataFrame(
    historical_prices, columns=["timestamp", "open", "high", "low", "close", "volume"]
)
historical_prices_df.tail()

,timestamp,open,high,low,close,volume
715,1615248000000,52383.0,54929.0,51875.0,54929.0,5890.742509
716,1615334400000,54922.7,57378.0,53051.0,55851.0,8091.884009
717,1615420800000,55859.9,58150.1,54286.3,57809.9,6111.043501
718,1615507200000,57807.0,58069.5,55040.2,57234.0,4971.615426
719,1615593600000,57238.9,60659.9,56073.2,60558.6,5839.766907


In [45]:
historical_prices_df["date"] = pd.to_datetime(
    historical_prices_df["timestamp"], unit="ms"
)
historical_prices_df.tail()

,timestamp,open,high,low,close,volume,date
715,1615248000000,52383.0,54929.0,51875.0,54929.0,5890.742509,2021-03-09
716,1615334400000,54922.7,57378.0,53051.0,55851.0,8091.884009,2021-03-10
717,1615420800000,55859.9,58150.1,54286.3,57809.9,6111.043501,2021-03-11
718,1615507200000,57807.0,58069.5,55040.2,57234.0,4971.615426,2021-03-12
719,1615593600000,57238.9,60659.9,56073.2,60558.6,5839.766907,2021-03-13


In [50]:
historical_prices_df.drop(columns="timestamp")
historical_prices_df.tail()

,timestamp,open,high,low,close,volume,date
715,1615248000000,52383.0,54929.0,51875.0,54929.0,5890.742509,2021-03-09
716,1615334400000,54922.7,57378.0,53051.0,55851.0,8091.884009,2021-03-10
717,1615420800000,55859.9,58150.1,54286.3,57809.9,6111.043501,2021-03-11
718,1615507200000,57807.0,58069.5,55040.2,57234.0,4971.615426,2021-03-12
719,1615593600000,57238.9,60659.9,56073.2,60558.6,5839.766907,2021-03-13


In [51]:
signals_df = historical_prices_df.loc[:, ["date", "open", "high", "low", "close", "volume"]].copy()
signals_df.tail()

,date,open,high,low,close,volume
715,2021-03-09,52383.0,54929.0,51875.0,54929.0,5890.742509
716,2021-03-10,54922.7,57378.0,53051.0,55851.0,8091.884009
717,2021-03-11,55859.9,58150.1,54286.3,57809.9,6111.043501
718,2021-03-12,57807.0,58069.5,55040.2,57234.0,4971.615426
719,2021-03-13,57238.9,60659.9,56073.2,60558.6,5839.766907


In [52]:
# Set the short window and long windows
short_window = 50
long_window = 100

# Set the `date` column as the index
signals_df = signals_df.set_index("date", drop=True)

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df["SMA50"] = signals_df["close"].rolling(window=short_window).mean()
signals_df["SMA100"] = signals_df["close"].rolling(window=long_window).mean()

# Initialize the new `Signal` column
signals_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA50 is under the SMA100, and
# where 1 is when the SMA50 is higher (or crosses over) the SMA100
signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA50"][short_window:] > signals_df["SMA100"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

# Print the DataFrame
signals_df.tail()

,open,high,low,close,volume,SMA50,SMA100,Signal,Entry/Exit
date,,,,,,,,,
2021-03-09,52383.0,54929.0,51875.0,54929.0,5890.742509,43593.726,35283.948,1.0,0.0
2021-03-10,54922.7,57378.0,53051.0,55851.0,8091.884009,43992.234,35645.377,1.0,0.0
2021-03-11,55859.9,58150.1,54286.3,57809.9,6111.043501,44438.196,36035.647,1.0,0.0
2021-03-12,57807.0,58069.5,55040.2,57234.0,4971.615426,44966.224,36415.673,1.0,0.0
2021-03-13,57238.9,60659.9,56073.2,60558.6,5839.766907,45517.396,36826.746,1.0,0.0
